In [67]:
from pysat.formula import CNF
from pysat.solvers import *
import re
import pandas as pd
import os

CONFIG_DIR = "../linux-4.14.152"
DIMACS_FLA = 'examples/out.dimacs'
CONFIG = "../linux-4.14.152/.config"
CSV = 'examples/alloptions-x86.4.14.152.csv'
CONF_FLAG = ["randconfig"] # , "defconfig", "allnoconfig", "allyesconfig", "allmodconfig", "alldefconfig"]

In [81]:
variables = {}
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c"):
            m = re.search('(c) (\d+) (\w+)', line)
            var_id = int(m.group(2))
            var_name = m.group(3)  
            variables[var_id] = var_name

def search_kconfig_type(option_name, df_options):
    ktype = df_options.query("option == " + '"' + option_name + '"')['type']
    return ktype.values[0]

# we are thinking the DIMACS id (number k) corresponding to the "module" variable of an option
def search_kmodule(option_name):
    for k, v in variables.items():
        if v == option_name + "_MODULE":
            return k

def get_dict(filename):
    """Dictionary from a linux config file"""
    res = dict()
    with open(CONFIG, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res

def get_var(option):
    for k, v in variables.items():
        if v == option:
            return k
        
def tristate_repartition(config_dict):
    return {'yes' : list(config_dict.values()).count('y'),
            'mod' : list(config_dict.values()).count('m')}
        

In [89]:
def check_both(config_dict):
    fla = CNF(DIMACS_FLA)
    # we read once and for all
    df_options = pd.read_csv(CSV)
    assumptions = []
    nb_yes = 0
    nb_mod = 0
    with Solver(bootstrap_with=fla.clauses) as l:
        print("at the beginning, fla is", l.solve(assumptions=[]))
        print("=> ADDING FEATURES FROM THE .CONFIG FILE [BOOL&TRISTATE]")
        for k, v in variables.items():
            if v in config_dict:
                if (v.endswith("_MODULE") or "CHOICE_" in v):  
                    continue # ignore
                kconfig_type = search_kconfig_type(v, df_options)
                if (kconfig_type == 'TRISTATE'):
                    # we have to find the other "module" variable that emulates the "m" value 
                    kmodule = search_kmodule(v)
                    # YES
                    if config_dict[v] == 'y':
                        print("+ Adding {} ({})".format(v, k))
                        print("+ Adding {} ({})".format(v + "_MODULE", -kmodule))
                        assumptions.extend([k, -kmodule])
                        nb_yes += 1
                    # MODULE
                    elif config_dict[v] == 'm':
                        print("+ Adding {} ({})".format(v, -k))
                        print("+ Adding {} ({})".format(v + "_MODULE", kmodule))
                        assumptions.extend([-k, kmodule])
                        nb_mod += 1
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                                "last_clause" : v,
                                "in" : True,
                                "core" : core}
                elif (kconfig_type == 'BOOL'):
                    # we have to find the other "module" variable that emulates the "m" value 
                    kmodule = search_kmodule(v)
                    print("+ Adding {} ({})".format(v, k))
                    assumptions.extend([k])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                                "last_clause" : v,
                                "in" : True,
                                "core" : core}
        print("=> ADDING FEATURES THAT ARE NOT IN THE .CONFIG FILE [BOOL&TRISTATE]")
        for f in df_options.option:
            # Feature in the CSV and in the DIMACS but not in the config
            if f not in config and f in variables.values():
                k = get_var(f)
                kconfig_type = search_kconfig_type(f, df_options)
                if (kconfig_type == 'TRISTATE'):
                    kmodule = search_kmodule(f)
                    # NO
                    print("+ Adding {} ({})".format(f, -k))
                    print("+ Adding {} ({})".format(f + "_MODULE", -kmodule))
                    assumptions.extend([-k, -kmodule])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                            "last_clause" : f,
                            "in" : True,
                            "core" : core}
                elif (kconfig_type == 'BOOL'):
                    kmodule = search_kmodule(f)
                    # NO
                    print("+ Adding {} ({})".format(f, -k))
                    assumptions.extend([-k])
                    # SAT Testing
                    if not l.solve(assumptions=assumptions):
                        print("== UNSAT")
                        core = list(map((lambda x : '~' + variables[abs(x)] if (x < 0) else variables[x]),
                                        l.get_core()))
                        print(core)
                        return {"sat" : False,
                            "last_clause" : f,
                            "in" : True,
                            "core" : core}
        print("== SAT")
        return {"sat" : False,
                "last_clause" : None,
                "in" : True,
                "core" : None}
    
def checker(n):
    stream = open("examples/extractor_out.csv", "w")
    stream.write("name,sat,nb_yes,nb_mod,last_clause,in,core\n")
    for i in range(n):
        for flag in CONF_FLAG:
            os.system("make {} -C {}".format(flag, CONFIG_DIR))
            config_dict = get_dict(CONFIG)
            tristate_nb = tristate_repartition(config_dict)
            res_checker = check_both(config_dict)
            name = "{}_{}".format(flag, i)
            core = None
            if res_checker["core"] is not None:
                core = "{}".format(res_checker["core"]).replace(',', ' ')
            stream.write("{},{},{},{},{},{},{}\n".format(name, res_checker["sat"], tristate_nb["yes"],
                                                      tristate_nb["mod"], res_checker["last_clause"],
                                                      res_checker["in"], core))
    stream.close()

In [ ]:
checker(100)

at the beginning, fla is True
=> ADDING FEATURES FROM THE .CONFIG FILE [BOOL&TRISTATE]
+ Adding X86_CMPXCHG64 (23722)
+ Adding NLS_MAC_GAELIC (54865)
+ Adding NLS_MAC_GAELIC_MODULE (-54864)
+ Adding SENSORS_MAX1668 (36161)
+ Adding SENSORS_MAX1668_MODULE (-36160)
+ Adding FB_MATROX_I2C (7472)
+ Adding FB_MATROX_I2C_MODULE (-7473)
+ Adding SRAM (15364)
+ Adding NET_ACT_SKBEDIT (46590)
+ Adding NET_ACT_SKBEDIT_MODULE (-46589)
+ Adding SERIAL_ARC (27267)
+ Adding SERIAL_ARC_MODULE (-27266)
+ Adding HAVE_DMA_CONTIGUOUS (10160)
+ Adding CRYPTO_GHASH_CLMUL_NI_INTEL (34003)
+ Adding CRYPTO_GHASH_CLMUL_NI_INTEL_MODULE (-34002)
+ Adding FB_SIS_300 (37125)
+ Adding FB_SM501 (11266)
+ Adding FB_SM501_MODULE (-11265)
+ Adding DMA_VIRTUAL_CHANNELS (37743)
+ Adding DMA_VIRTUAL_CHANNELS_MODULE (-37742)
+ Adding HAVE_RCU_TABLE_FREE (10031)
+ Adding CRYPTO_CRYPTD (21863)
+ Adding CRYPTO_CRYPTD_MODULE (-21864)
+ Adding AMIGA_PARTITION (38100)
+ Adding X86_DIRECT_GBPAGES (40107)
+ Adding USB_OHCI_HCD_PCI

+ Adding USB_SERIAL_WISHBONE (54423)
+ Adding USB_SERIAL_WISHBONE_MODULE (-54422)
+ Adding NVMEM_LPC18XX_EEPROM (12793)
+ Adding NVMEM_LPC18XX_EEPROM_MODULE (-12792)
+ Adding PERSISTENT_KEYRINGS (38802)
+ Adding XILLYBUS (7164)
+ Adding XILLYBUS_MODULE (-7163)
+ Adding FUSE_FS (36776)
+ Adding FUSE_FS_MODULE (-36775)
+ Adding FB_VGA16 (24215)
+ Adding FB_VGA16_MODULE (-24214)
+ Adding RING_BUFFER (14622)
+ Adding USB_SERIAL_OTI6858 (39380)
+ Adding USB_SERIAL_OTI6858_MODULE (-39379)
+ Adding COMMON_CLK_BOSTON (15847)
+ Adding DMA_ENGINE (2580)
+ Adding USB_COMMON (56818)
+ Adding USB_COMMON_MODULE (-56817)
+ Adding AHCI_XGENE (43625)
+ Adding AHCI_XGENE_MODULE (-43624)
+ Adding W1 (10)
+ Adding W1_MODULE (-8)
+ Adding PSAMPLE (99)
+ Adding PSAMPLE_MODULE (-98)
+ Adding FB_HGA (39420)
+ Adding FB_HGA_MODULE (-39419)
+ Adding MTD_REDBOOT_PARTS_READONLY (59344)
+ Adding SBC_EPX_C3_WATCHDOG (4848)
+ Adding SBC_EPX_C3_WATCHDOG_MODULE (-4847)
+ Adding SENSORS_TSL2550 (42383)
+ Adding SENSORS

+ Adding SUNXI_CCU (17605)
+ Adding HPFS_FS (20336)
+ Adding HPFS_FS_MODULE (-20335)
+ Adding THREAD_INFO_IN_TASK (209)
+ Adding SND_DRIVERS (7228)
+ Adding CRYPTO_USER_API_SKCIPHER (48893)
+ Adding CRYPTO_USER_API_SKCIPHER_MODULE (-48894)
+ Adding LEDS_DAC124S085 (1480)
+ Adding LEDS_DAC124S085_MODULE (-1479)
+ Adding SERIAL_8250_DEPRECATED_OPTIONS (74392)
+ Adding I2C_CHARDEV (76360)
+ Adding I2C_CHARDEV_MODULE (-76359)
+ Adding T5403 (4462)
+ Adding T5403_MODULE (-4461)
+ Adding SENSORS_PWM_FAN (65912)
+ Adding SENSORS_PWM_FAN_MODULE (-65911)
+ Adding FB_TMIO_ACCELL (34100)
+ Adding REGULATOR_DA9063 (10137)
+ Adding REGULATOR_DA9063_MODULE (-10136)
+ Adding I2C_VIA (10030)
+ Adding I2C_VIA_MODULE (-10029)
+ Adding REED_SOLOMON_ENC8 (10957)
+ Adding GRO_CELLS (48870)
+ Adding MMC_SPI (3052)
+ Adding MMC_SPI_MODULE (-3043)
+ Adding SSB_SDIOHOST (7467)
+ Adding MAC802154 (16777)
+ Adding MAC802154_MODULE (-16776)
+ Adding SSB_PCIHOST (28357)
+ Adding DMARD10 (70232)
+ Adding DMARD10_MO

+ Adding XZ_DEC_ARMTHUMB (44757)
+ Adding AD5446 (227)
+ Adding AD5446_MODULE (-226)
+ Adding HFSPLUS_FS (56577)
+ Adding HFSPLUS_FS_MODULE (-56578)
+ Adding MFD_SMSC (13185)
+ Adding LIBFDT (44576)
+ Adding SERIAL_ATMEL_TTYAT (70967)
+ Adding HAS_IOPORT_MAP (11666)
+ Adding X86_FAST_FEATURE_TESTS (59578)
+ Adding PCIEASPM (22752)
+ Adding SCHED_SMT (67427)
+ Adding GENWQE (15047)
+ Adding GENWQE_MODULE (-15053)
+ Adding USB_MUSB_HDRC (421)
+ Adding USB_MUSB_HDRC_MODULE (-422)
+ Adding FB (415)
+ Adding FB_MODULE (-432)
+ Adding CHR_DEV_SCH (25268)
+ Adding CHR_DEV_SCH_MODULE (-25267)
+ Adding MAX5481 (76312)
+ Adding MAX5481_MODULE (-76311)
+ Adding USB_FOTG210_HCD (74399)
+ Adding USB_FOTG210_HCD_MODULE (-74398)
+ Adding FIREWIRE_NOSY (21811)
+ Adding FIREWIRE_NOSY_MODULE (-21810)
+ Adding MFD_LM3533 (6011)
+ Adding MFD_LM3533_MODULE (-6012)
+ Adding NVME_FABRICS (24845)
+ Adding NVME_FABRICS_MODULE (-24844)
+ Adding REGULATOR_PBIAS (50667)
+ Adding REGULATOR_PBIAS_MODULE (-50666)
+ 

+ Adding DEBUG_TLBFLUSH (48637)
+ Adding MANDATORY_FILE_LOCKING (6248)
+ Adding NO_HZ_IDLE (8809)
+ Adding SCSI_DMA (39421)
+ Adding NET_9P_XEN (53202)
+ Adding NET_9P_XEN_MODULE (-53201)
+ Adding IIO_SYSFS_TRIGGER (9723)
+ Adding IIO_SYSFS_TRIGGER_MODULE (-9722)
+ Adding INTEL_MEI_WDT (23981)
+ Adding INTEL_MEI_WDT_MODULE (-23982)
+ Adding I2C_PIIX4 (24219)
+ Adding I2C_PIIX4_MODULE (-24218)
+ Adding TRACE_CLOCK (60268)
+ Adding OF (780)
+ Adding GENERIC_ADC_BATTERY (43130)
+ Adding GENERIC_ADC_BATTERY_MODULE (-43129)
+ Adding FB_SYS_FILLRECT (57076)
+ Adding FB_SYS_FILLRECT_MODULE (-57075)
+ Adding REMOTEPROC (3593)
+ Adding REMOTEPROC_MODULE (-3594)
+ Adding MAX1118 (52372)
+ Adding MAX1118_MODULE (-52373)
+ Adding MFD_STM32_LPTIMER (12446)
+ Adding MFD_STM32_LPTIMER_MODULE (-12447)
+ Adding SND_HWDEP (61910)
+ Adding SND_HWDEP_MODULE (-61909)
+ Adding GENERIC_ALLOCATOR (26576)
+ Adding USB_OTG_FSM (12687)
+ Adding USB_OTG_FSM_MODULE (-12686)
+ Adding TIMERFD (50958)
+ Adding REGULA

...

+ Adding NLS_CODEPAGE_775 (66494)
+ Adding NLS_CODEPAGE_775_MODULE (-66493)
+ Adding CRYPTO_CMAC (19746)
+ Adding CRYPTO_CMAC_MODULE (-19745)
+ Adding GPIO_WM831X (53498)
+ Adding GPIO_WM831X_MODULE (-53497)
+ Adding NOTIFIER_ERROR_INJECTION (30108)
+ Adding NOTIFIER_ERROR_INJECTION_MODULE (-30107)
+ Adding NSC_GPIO (65582)
+ Adding NSC_GPIO_MODULE (-65581)
+ Adding ISA_DMA_API (4962)
+ Adding CRYPTO_MD4 (53442)
+ Adding CRYPTO_MD4_MODULE (-53441)
+ Adding I2C_IMG (7056)
+ Adding I2C_IMG_MODULE (-7055)
+ Adding LOGO_LINUX_CLUT224 (26618)
+ Adding SND_SUPPORT_OLD_API (71854)
+ Adding THIN_ARCHIVES (15659)
+ Adding GPIO_TS4900 (13238)
+ Adding GPIO_TS4900_MODULE (-13239)
+ Adding REGULATOR_RK808 (35468)
+ Adding REGULATOR_RK808_MODULE (-35467)
+ Adding IRQ_DOMAIN_HIERARCHY (5945)
+ Adding BACKLIGHT_SKY81452 (4809)
+ Adding BACKLIGHT_SKY81452_MODULE (-4808)
+ Adding LEDS_LP55XX_COMMON (62142)
+ Adding LEDS_LP55XX_COMMON_MODULE (-62141)
+ Adding SYSC_RCAR (24794)
+ Adding HW_RANDOM_VIA (35

+ Adding REGULATOR_LTC3676 (13027)
+ Adding REGULATOR_LTC3676_MODULE (-13028)
+ Adding X86_TSC (9365)
+ Adding HVC_DRIVER (93)
+ Adding CRYPTO_BLKCIPHER (32453)
+ Adding CRYPTO_BLKCIPHER_MODULE (-32454)
+ Adding RESET_SOCFPGA (2813)
+ Adding SLUB_DEBUG (2795)
+ Adding DRM_DEBUG_MM (10514)
+ Adding SENSORS_DELL_SMM (35090)
+ Adding SENSORS_DELL_SMM_MODULE (-35089)
+ Adding THERMAL (3303)
+ Adding THERMAL_MODULE (-3302)
+ Adding CLEANCACHE (39981)
+ Adding PHY_RALINK_USB (188)
+ Adding PHY_RALINK_USB_MODULE (-187)
+ Adding FB_BROADSHEET (3231)
+ Adding FB_BROADSHEET_MODULE (-3232)
+ Adding DECOMPRESS_LZMA (39404)
+ Adding DECOMPRESS_LZMA_MODULE (-39403)
+ Adding ARCH_WANT_IPC_PARSE_VERSION (494)
+ Adding SENSORS_NCT7802 (11295)
+ Adding SENSORS_NCT7802_MODULE (-11294)
+ Adding COMMON_RESET_HI3660 (67755)
+ Adding COMMON_RESET_HI3660_MODULE (-67754)
+ Adding RESET_HSDK (66905)
+ Adding MMC (45)
+ Adding MMC_MODULE (-48)
+ Adding REGULATOR_MT6323 (53564)
+ Adding REGULATOR_MT6323_MODULE (-

+ Adding IB700_WDT (54455)
+ Adding IB700_WDT_MODULE (-54454)
+ Adding PHY_BCM_NS_USB2 (25777)
+ Adding PHY_BCM_NS_USB2_MODULE (-25776)
+ Adding HAVE_KERNEL_BZIP2 (405)
+ Adding SND_PCM_ELD (69782)
+ Adding MTD_L440GX (7152)
+ Adding MTD_L440GX_MODULE (-7151)
+ Adding CRYPTO_GLUE_HELPER_X86 (30270)
+ Adding CRYPTO_GLUE_HELPER_X86_MODULE (-30269)
+ Adding MFD_MAX8925 (146)
+ Adding UNIPHIER_WATCHDOG (23914)
+ Adding UNIPHIER_WATCHDOG_MODULE (-23915)
+ Adding LOGO (26615)
+ Adding TRANSPARENT_HUGEPAGE_ALWAYS (47897)
+ Adding NLS_KOI8_U (43292)
+ Adding NLS_KOI8_U_MODULE (-43291)
+ Adding BLK_DEV_LOOP (594)
+ Adding BLK_DEV_LOOP_MODULE (-593)
+ Adding SENSORS_LM25066 (74069)
+ Adding SENSORS_LM25066_MODULE (-74068)
+ Adding NLS_ISO8859_3 (59863)
+ Adding NLS_ISO8859_3_MODULE (-59862)
+ Adding RESET_LPC18XX (49677)
+ Adding SENSORS_INA2XX (383)
+ Adding SENSORS_INA2XX_MODULE (-382)
+ Adding USB_F_MASS_STORAGE (3722)
+ Adding USB_F_MASS_STORAGE_MODULE (-3721)
+ Adding MFD_SEC_CORE (3493)
+ 

+ Adding SPARSE_IRQ (20026)
+ Adding MTD_PARSER_TRX (34555)
+ Adding MTD_PARSER_TRX_MODULE (-34554)
+ Adding TICK_CPU_ACCOUNTING (47708)
+ Adding RESET_ZX2967 (61486)
+ Adding HTC_PASIC3 (52083)
+ Adding HTC_PASIC3_MODULE (-52082)
+ Adding GPIO_ATH79 (11196)
+ Adding GPIO_ATH79_MODULE (-11197)
+ Adding SND_TIMER (25865)
+ Adding SND_TIMER_MODULE (-25864)
+ Adding EEPROM_LEGACY (38406)
+ Adding EEPROM_LEGACY_MODULE (-38405)
+ Adding EXT3_FS_SECURITY (46708)
+ Adding NLS_ISO8859_13 (63601)
+ Adding NLS_ISO8859_13_MODULE (-63600)
+ Adding NVMEM_BCM_OCOTP (65006)
+ Adding NVMEM_BCM_OCOTP_MODULE (-65005)
+ Adding USB_MDC800 (27605)
+ Adding USB_MDC800_MODULE (-27604)
+ Adding REGULATOR_DA9211 (13120)
+ Adding REGULATOR_DA9211_MODULE (-13121)
+ Adding MWAVE (51723)
+ Adding MWAVE_MODULE (-51722)
+ Adding TRACING_SUPPORT (2465)
+ Adding DEBUG_ENTRY (56374)
+ Adding HAS_IOMEM (11)
+ Adding MTD_GEN_PROBE (9580)
+ Adding MTD_GEN_PROBE_MODULE (-9581)
+ Adding DRM_SII902X (13250)
+ Adding DRM_SII9

+ Adding USB_STORAGE_DATAFAB (34464)
+ Adding USB_STORAGE_DATAFAB_MODULE (-34463)
+ Adding FB_OLPC_DCON (17123)
+ Adding FB_OLPC_DCON_MODULE (-17122)
+ Adding RN5T618_WATCHDOG (23952)
+ Adding RN5T618_WATCHDOG_MODULE (-23953)
+ Adding ARCH_SUSPEND_POSSIBLE (9572)
+ Adding USB_R8A66597_HCD (75924)
+ Adding USB_R8A66597_HCD_MODULE (-75923)
+ Adding CRYPTO_SHA1 (3802)
+ Adding CRYPTO_SHA1_MODULE (-3801)
+ Adding USB_CHIPIDEA_HOST (29603)
+ Adding CRC4 (51391)
+ Adding CRC4_MODULE (-51390)
+ Adding DRM_MEGACHIPS_STDPXXXX_GE_B850V3_FW (46219)
+ Adding DRM_MEGACHIPS_STDPXXXX_GE_B850V3_FW_MODULE (-46218)
+ Adding ATMEL_SSC (10922)
+ Adding ATMEL_SSC_MODULE (-10921)
+ Adding BLK_SCSI_REQUEST (27501)
+ Adding FREEZER (45804)
+ Adding BCM_KONA_TIMER (34363)
+ Adding GPIO_IOP (15661)
+ Adding GPIO_IOP_MODULE (-15660)
+ Adding FS_IOMAP (57522)
+ Adding XZ_DEC_SPARC (49790)
+ Adding ROCKCHIP_GRF (19101)
+ Adding CRYPTO_CRCT10DIF (54977)
+ Adding CRYPTO_CRCT10DIF_MODULE (-54976)
+ Adding CRYPTO_RNG 

[1, 2, 3]

False